In [1]:
import IPython.display
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm

import tensorflow as tf

import nengo
from nengo.utils.matplotlib import rasterplot
from nengo.dists import Choice
from nengo.utils.ensemble import tuning_curves

import hgtk
 
%matplotlib inline

In [2]:
total_cho = hgtk.josa.CHO
total_joong = hgtk.josa.JOONG
total_jong = hgtk.josa.JONG
print(total_cho)
print(total_joong)
print(total_jong)

('ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ')
('ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ')
('', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ')


In [3]:
ori_sent = '아 더빙.. 진짜 짜증나네요 목소리'
import hgtk
 
jamo_sent = hgtk.text.decompose(ori_sent)
jamo_sent = jamo_sent.replace('ᴥ', '')
print(jamo_sent + '\n')

many_hot = np.zeros(len(total_cho + total_joong + total_jong))
sent_hot = np.zeros((len(ori_sent),len(many_hot)))
word_idx = 0

for word in ori_sent:
    #print("input_word: " + str(word))
    if(hgtk.checker.is_hangul(word)):
        split_word = hgtk.letter.decompose(word)
        cho_idx = 0
        joong_idx = 0
        jong_idx = 0
        one_hot_cho = np.zeros(len(hgtk.josa.CHO))
        one_hot_joong = np.zeros(len(hgtk.josa.JOONG))
        one_hot_jong = np.zeros(len(hgtk.josa.JONG))
        if(len(split_word) > 2):
            for cho in total_cho:
                if(cho == split_word[0]):
                    one_hot_cho[cho_idx] = 1
                    break
                else:
                    cho_idx+= 1
            for joong in total_joong:
                if(joong == split_word[1]):
                    one_hot_joong[joong_idx] = 1
                    break
                else:
                    joong_idx+= 1
            for jong in total_jong:
                if(jong == split_word[2]):
                    one_hot_jong[jong_idx] = 1
                    break
                else:
                    jong_idx+= 1
        else:
            for cho in total_cho:
                if(cho == split_word[0]):
                    one_hot_cho[cho_idx] = 1
                    break
                else:
                    cho_idx+= 1
            for joong in total_joong:
                if(joong == split_word[1]):
                    one_hot_joong[joong_idx] = 1
                    break
                else:
                    joong_idx+= 1
        many_hot = np.append(one_hot_cho, one_hot_joong)
        many_hot = np.append(many_hot, one_hot_jong)
        
    else:
        many_hot = np.zeros(len(total_cho + total_joong + total_jong))
    sent_hot[word_idx] = many_hot
    word_idx += 1


print(sent_hot)
print(len(sent_hot))
print(len(ori_sent))
print(len(sent_hot[0]))
print(sent_hot[0])

ㅇㅏ ㄷㅓㅂㅣㅇ.. ㅈㅣㄴㅉㅏ ㅉㅏㅈㅡㅇㄴㅏㄴㅔㅇㅛ ㅁㅗㄱㅅㅗㄹㅣ

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
19
19
68
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [4]:
time_duration = 10

In [ ]:
def sent_input(t):
    ms = int(t * 1000)
    char_num = int(ms/(time_duration / len(ori_sent))-1)
    voice = sent_hot[char_num]
    return voice

In [5]:
def answer_input(t):
    ms = int(t * 1000)
    